In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import librosa
import librosa.display

In [8]:
import pandas as pd

In [9]:
meta=pd.read_csv('esc50.csv')

In [10]:
meta.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [11]:
meta['category'].value_counts()

dog                 40
glass_breaking      40
drinking_sipping    40
rain                40
insects             40
laughing            40
hen                 40
engine              40
breathing           40
crying_baby         40
hand_saw            40
coughing            40
snoring             40
chirping_birds      40
toilet_flush        40
pig                 40
washing_machine     40
clock_tick          40
sneezing            40
rooster             40
sea_waves           40
siren               40
cat                 40
door_wood_creaks    40
helicopter          40
crackling_fire      40
car_horn            40
brushing_teeth      40
vacuum_cleaner      40
thunderstorm        40
door_wood_knock     40
can_opening         40
crow                40
clapping            40
fireworks           40
chainsaw            40
airplane            40
mouse_click         40
pouring_water       40
train               40
sheep               40
water_drops         40
church_bells        40
clock_alarm

In [12]:
import os
import numpy as np

In [13]:
audio_file='C:\\Users\\aneri\\OneDrive\\Desktop\\ESC-50-master\\audio'

In [14]:
audio_file

'C:\\Users\\aneri\\OneDrive\\Desktop\\ESC-50-master\\audio'

In [15]:
def feature_extractor(file):
    audio, sample_rate=librosa.load(file_name,res_type='kaiser_fast')
    mfcc_features=librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=40)
    mfcc_scaled=np.mean(mfcc_features.T,axis=0)
    return mfcc_scaled

In [16]:
from tqdm import tqdm

In [17]:
extracted_feature=[]
for index_num,row in tqdm(meta.iterrows()):
    file_name=os.path.join(os.path.abspath(audio_file),str(row['filename']))
    final_class=row['category']
    data=feature_extractor(file_name)
    extracted_feature.append([data,final_class])

2000it [01:37, 20.46it/s]


In [18]:
df=pd.DataFrame(extracted_feature,columns=['features','class'])

In [19]:
df

,features,class
0,"[-601.3449, 5.261408, -9.065388, -3.7297592, -...",dog
1,"[-200.98369, 12.612191, -69.114265, 13.922092,...",chirping_birds
2,"[7.305005, 69.53838, -21.44031, 27.377811, -24...",vacuum_cleaner
3,"[8.73106, 67.60287, -19.778683, 26.026417, -21...",vacuum_cleaner
4,"[-428.58066, 132.67262, 31.145851, 42.445656, ...",thunderstorm
...,...,...
1995,"[-212.94466, 128.4998, -58.266853, 7.451922, -...",hen
1996,"[-60.192314, 39.280502, -18.384956, 9.533246, ...",vacuum_cleaner
1997,"[-290.72827, 86.82194, 5.8481264, 23.00583, 3....",footsteps
1998,"[-155.02011, 104.69687, -47.293575, 21.655563,...",sheep


In [38]:
X=np.array(df['features'].tolist())
X.shape

(2000, 40)

In [21]:
y=np.array(df['class'].tolist())
y.shape

(2000,)

In [22]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))
y.shape

(2000, 50)

In [43]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [44]:
X_train.shape

(1600, 40)

In [45]:
X_test.shape

(400, 40)

In [46]:
y_train.shape

(1600, 50)

In [47]:
y_test.shape

(400, 50)

In [48]:
import tensorflow as tf

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [50]:
model=Sequential()
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(50))
model.add(Activation('softmax'))

In [51]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               4100      
                                                                 
 activation_4 (Activation)   (None, 100)               0         
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 200)               20200     
                                                                 
 activation_5 (Activation)   (None, 200)               0         
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_6 (Dense)             (None, 200)              

In [52]:
#model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [53]:
from tensorflow import keras
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [54]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 
checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5',verbose=1, save_best_only=True)

start = datetime.now()

model.fit(X_train, y_train, batch_size=32, epochs=500, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/500
24/50 [=============>................] - ETA: 0s - loss: 51.6739 - accuracy: 0.0091     
Epoch 1: val_loss improved from inf to 4.06514, saving model to saved_models\audio_classification.hdf5
50/50 [==============================] - 1s 6ms/step - loss: 37.3372 - accuracy: 0.0113 - val_loss: 4.0651 - val_accuracy: 0.0175
Epoch 2/500
23/50 [============>.................] - ETA: 0s - loss: 14.4742 - accuracy: 0.0340
Epoch 2: val_loss improved from 4.06514 to 3.87594, saving model to saved_models\audio_classification.hdf5
50/50 [==============================] - 0s 3ms/step - loss: 12.2087 - accuracy: 0.0294 - val_loss: 3.8759 - val_accuracy: 0.0225
Epoch 3/500
30/50 [=================>............] - ETA: 0s - loss: 8.0083 - accuracy: 0.0292 
Epoch 3: val_loss did not improve from 3.87594
50/50 [==============================] - 0s 2ms/step - loss: 7.4389 - accuracy: 0.0312 - val_loss: 3.9041 - val_accuracy: 0.0150
Epoch 4/500
30/50 [=================>............] - ETA: 0s 

In [57]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.375
